In [2]:
!pip install xmlschema==1.0.18
!pip install pyarrow==3.0.0

     |████████████████████████████████| 358 kB 28.0 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 16.3 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 20.7 MB 23.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import boto3
import xmlschema
from concurrent.futures import ThreadPoolExecutor 

In [5]:
s3 = boto3.client('s3')
s3.download_file('wos-bucket', 'xml_to_parquet.py', 'xml_to_parquet.py')
s3.download_file('wos-bucket', 'convert_xml_to_parquet.py', 'convert_xml_to_parquet.py')
s3.download_file('wos-bucket', 'WR_2023_20240112181857_CORE_0001.xml', 'WR_2023_20240112181857_CORE_0001.xml')
s3.download_file('wos-bucket', 'WR_2023_20240112181857_CORE_0002.xml', 'WR_2023_20240112181857_CORE_0002.xml')
s3.download_file('wos-bucket', 'WR_2023_20240112181857_CORE_0003.xml', 'WR_2023_20240112181857_CORE_0003.xml')

In [7]:
schemas = ['item_dii.rawxml.xsd',
 'item_cabi.rawxml.xsd',
 'item_ccc.rawxml.xsd',
 'BIOSIS_common.rawxml.xsd',
 'item_bp.rawxml.xsd',
 'CHANGELOG',
 'item_bci.rawxml.xsd',
 'summary.rawxml.xsd',
 'fullrecord_metadata.rawxml.xsd',
 'item_medline.rawxml.xsd',
 'item_ba.rawxml.xsd',
 'item_wos.rawxml.xsd',
 'common_types.rawxml.xsd',
 'item_zr.rawxml.xsd',
 'clarivate.com.schema.wok5.30.rawxml.xsd',
 'EWUID.rawxml.xsd',
 'item_fsta.rawxml.xsd',
 'item_rsci.rawxml.xsd',
 'item_drci.rawxml.xsd',
 'item_cscd.rawxml.xsd',
 'item_inspec.rawxml.xsd',
 'item_scielo.rawxml.xsd']

for xsd in schemas:
    s3.download_file('wos-bucket', xsd, xsd)

In [22]:
# Test xml_to_parquet
s3.download_file('wos-bucket', 'PurchaseOrder.xml', 'PurchaseOrder.xml')
s3.download_file('wos-bucket', 'PurchaseOrder.xsd', 'PurchaseOrder.xsd')
!python xml_to_parquet.py -x PurchaseOrder.xsd PurchaseOrder.xml

INFO - 2024-05-24 15:07:49 - Parsing XML Files..
INFO - 2024-05-24 15:07:49 - Processing 1 files
DEBUG - 2024-05-24 15:07:49 - Generating schema from PurchaseOrder.xsd
DEBUG - 2024-05-24 15:07:49 - Parsing PurchaseOrder.xml
DEBUG - 2024-05-24 15:07:49 - Saving to: /home/jovyan/PurchaseOrder.xml.parquet
DEBUG - 2024-05-24 15:07:49 - Completed PurchaseOrder.xml


In [41]:
# Module won't run with WoS XML
# I suspect it's a schema issue; the test example uses a single xsd file,
# while clarivate.com.schema.wok5.30.rawxml.xsd references several
# supplementary xsd files
!python xml_to_parquet.py -x clarivate.com.schema.wok5.30.rawxml.xsd WR_2023_20240112181857_CORE_0001.xml -l out.txt -v

INFO - 2024-05-24 16:19:08 - Parsing XML Files..
INFO - 2024-05-24 16:19:08 - Processing 1 files
DEBUG - 2024-05-24 16:19:08 - Generating schema from clarivate.com.schema.wok5.30.rawxml.xsd
DEBUG - 2024-05-24 16:19:10 - Parsing WR_2023_20240112181857_CORE_0001.xml


In [ ]:
# This is the code should convert the XML files to Parquet files in parallel
# once the schema is correctly specified
schema = 'TBD'
!python xml_to_parquet.py -m 3 -x schema *.xml

In [ ]:
# This code should write the Parquet files to S3 in parallel
s3r = boto3.resource('s3', region_name = 'us-east-1')

par_files = []

def to_s3(file):
    body = open(file, 'rb')
        s3r.Bucket('wos-bucket').put_object(
          Key = file,
          Body = body)
        
with ThreadPoolExecutor(max_workers = 3) as executor:
    results = executor.map(to_s3, par_files)

    